In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
import random
# import xgboost as xgb
import gensim
import itertools
import time
import multiprocessing as mp
import copy
%matplotlib inline

In [2]:
final_train_data = pd.read_csv("../Data/final_train_data.csv")
final_test_data = pd.read_csv("../Data/final_test_data.csv")
# final_test_pre_final_data = pd.read_csv("../Data/final_test_pre_final_data.csv")
for var in final_train_data.columns:
    if final_train_data[var].dtype == 'float64':
        final_train_data[var] = final_train_data[var].astype(np.float32)
    elif final_train_data[var].dtype == 'int64':
        final_train_data[var] = final_train_data[var].astype(np.int32)
for var in final_test_data.columns:
    if final_test_data[var].dtype == 'float64':
        final_test_data[var] = final_test_data[var].astype(np.float32)
    elif final_test_data[var].dtype == 'int64':
        final_test_data[var] = final_test_data[var].astype(np.int32)
# for var in final_test_pre_final_data.columns:
#     if final_test_pre_final_data[var].dtype == 'float64':
#         final_test_pre_final_data[var] = final_test_pre_final_data[var].astype(np.float32)
#     elif final_test_pre_final_data[var].dtype == 'int64':
#         final_test_pre_final_data[var] = final_test_pre_final_data[var].astype(np.int32)

print final_train_data.shape
list(final_train_data.columns)

(8605870, 176)


['user_id',
 'product_id',
 'user_product_purchase_count',
 'user_product_add_to_cart_order',
 'user_product_recency',
 'user_product_recency_order',
 'user_product_orders_since_last_order',
 'user_product_last_dow',
 'user_product_last_tod',
 'user_product_days_since_last_order',
 'user_product_mean_days_between_purchase',
 'user_product_std_days_between_purchase',
 'user_product_order_corr',
 'user_product_day_corr',
 'user_product_mean_streak_length',
 'user_product_max_streak_length',
 'user_product_last_streak_length',
 'user_product_num_streak',
 'user_product_in_streak',
 'user_product_count',
 'user_order_frequency',
 'user_order_frequency_std',
 'user_order_recency',
 'user_order_recency_norm',
 'user_day_kinetic_energy',
 'user_tod_kinetic_energy',
 'user_aisle_count',
 'user_department_count',
 'user_cluster_count',
 'user_product_kinetic_energy',
 'user_product_kinetic_energy_norm',
 'user_aisle_kinetic_energy',
 'user_department_kinetic_energy',
 'user_cluster_kinetic_ener

In [3]:
# final_train_data.is_organic = final_train_data.is_organic*final_train_data.user_organic_fraction
# final_train_data.is_gluten_free = final_train_data.is_gluten_free*final_train_data.user_gluten_free_fraction
# final_test_data.is_organic = final_test_data.is_organic*final_test_data.user_organic_fraction
# final_test_data.is_gluten_free = final_test_data.is_gluten_free*final_test_data.user_gluten_free_fraction

# final_train_data[final_train_data.is_gluten_free > 0.5].reordered.mean()

0.10169491171836853

In [3]:
final_train_data['user_products_per_order'] = final_train_data['user_product_count']*1.0/final_train_data['order_number']
final_test_data['user_products_per_order'] = final_test_data['user_product_count']*1.0/final_test_data['order_number']

In [4]:
aisles_best_before = pd.read_csv("../Data/aisles_best_before.csv")
aisles_best_before = aisles_best_before[['aisle_id', 'Best before']]
final_train_data = final_train_data.merge(aisles_best_before, on = ['aisle_id'], how = 'left')
final_test_data = final_test_data.merge(aisles_best_before, on = ['aisle_id'], how = 'left')
final_train_data['Best before'].fillna(365, inplace = True)
final_test_data['Best before'].fillna(365, inplace = True)

In [5]:
indep_vars = ['user_product_purchase_count','user_product_purchase_ratio',
              'user_product_add_to_cart_order','user_product_recency',
              'user_product_orders_since_last_order','user_product_days_since_last_order','user_product_count',
              'user_order_frequency','user_order_frequency_std','user_order_recency','user_day_kinetic_energy',
              'user_tod_kinetic_energy','user_aisle_count','user_department_count','product_user_cluster_count',
              'is_organic','user_products_per_order','user_product_mean_streak_length','user_product_max_streak_length',
              'user_product_last_streak_length','user_product_num_streak',
              'user_product_in_streak',
#               'product_name_idf_max',
#               'user_mean_product_name_idf_max',
              'user_product_kinetic_energy_norm',
              'user_mean_streak_length','user_fraction_product_with_no_reorder','user_aisle_daily_reorder_rate',
              'user_aisle_hourly_reorder_rate','user_product_mean_days_between_purchase',
              'user_product_std_days_between_purchase','product_mean_streak_length',
              'Best before','user_last_2_order_sim','user_last_n_order_sim',
              'user_num_orders_with_no_reorder',
              'user_fraction_orders_with_no_reorder',
              'user_organic_count','user_mean_product_name_idf','cluster_aisle_count',
              'user_organic_fraction',
              'user_order_recency_norm','product_recency_norm','aisle_recency_norm',
#               'is_gluten_free',
              'product_num_users_with_no_reorders',
              'user_num_product_with_no_reorder','product_fraction_users_with_no_reorders',
              'user_gluten_free_count','cluster_recency_norm',
#               'department_recency_norm',
              'user_gluten_free_fraction','user_gluten_free_recency', 'user_organic_recency',
              'product_order_recency','product_order_recency_norm',
#               'user_product_purchase_due',
              'user_product_last_dow','user_product_last_tod',
              'user_product_last_dow_diff','user_product_last_tod_diff',
              'user_last_order_dow_diff','user_last_order_tod_diff',
              'user_cluster_count',
              'user_product_kinetic_energy','user_aisle_kinetic_energy','user_department_kinetic_energy',
              'user_cluster_kinetic_energy','user_reorder_count_mean',
#               'user_reorder_frequency_mean',
              'user_reorder_count_last','user_cluster_mean_reorder_count',
#               'user_cluster_mean_product_counts',
              'user_reorder_count_std','user_reorder_fraction_mean','user_reorder_fraction_std',
              'user_reorder_fraction_last',
#               'posterior_probability_jj_daily','posterior_probability_jj_hourly',
#               'user_reorder_frequency_std',
#               'user_product_daily_order_std','user_product_hourly_order_std',
              'user_daily_reorder_rate','user_hourly_reorder_rate',
              'product_daily_reorder_rate',
              'product_hourly_reorder_rate',
              'user_product_daily_reorder_rate','user_product_hourly_reorder_rate',
              'user_mean_basket_size','user_std_basket_size','user_last_basket_size','user_tenure',
              'user_mean_product_name_length',
              'product_user_count','product_order_count',
#               'product_order_per_user',
              'product_dow_kinetic_energy','aisle_order_per_user','cluster_order_per_user','department_order_per_user',
              'product_tod_kinetic_energy',
#               'product_reorder_frequency',
              'product_mean_add_to_cart_order','user_product_order_corr',
#               'user_product_day_corr',
              'product_mean_days_between_orders','product_std_days_between_orders','product_idf',
              'product_name_length','product_name_idf',
              'user_aisle_purchase_count','user_aisle_orders_since_last_order','user_aisle_add_to_cart_order',
              'user_aisle_recency','user_department_purchase_count','user_department_orders_since_last_order',
              'user_department_add_to_cart_order',
              'user_department_recency','user_cluster_purchase_count',
              'user_cluster_orders_since_last_order','user_cluster_add_to_cart_order',
              'user_cluster_recency', 
              'order_number','order_dow','order_hour_of_day','days_since_prior_order','user_daily_order_rate',
              'user_hourly_order_rate','product_daily_order_rate',
              'product_hourly_order_rate',
              'user_product_daily_order_rate','user_product_hourly_order_rate',
#               'aisle_user_count',
              'aisle_order_recency','aisle_order_recency_norm',
#               'aisle_order_count',
              'aisle_dow_kinetic_energy','aisle_tod_kinetic_energy','aisle_reorder_frequency',
              'aisle_mean_add_to_cart_order','aisle_product_count',
#               'aisle_mean_days_between_orders',
#               'aisle_std_days_between_orders',
              'department_user_count','user_std_product_name_length','user_std_product_name_idf',
              'department_dow_kinetic_energy','department_tod_kinetic_energy','department_reorder_frequency',
              'department_mean_add_to_cart_order',
              'department_product_count',
#               'department_mean_days_between_orders',
#               'department_std_days_between_orders',
              'aisle_idf', 'user_product_recency_order',
              'cluster_idf',
#               'department_idf',
              'cluster_user_count','user_cluster_size',
#               'user_cluster',
              'user_cluster_reorder_rate',
#               'cluster_order_count',
              'cluster_dow_kinetic_energy','cluster_tod_kinetic_energy','cluster_reorder_frequency',
              'cluster_mean_add_to_cart_order','cluster_product_count',
#               'cluster_mean_days_between_orders',
#               'cluster_std_days_between_orders',
              'posterior_probability_product','posterior_probability_aisle',
              'posterior_probability_cluster','posterior_probability_department',
              'user_product_similarity',
              'product_none_similarity','product_recency',
              'aisle_recency',
              'cluster_recency','department_recency','cluster_order_recency','cluster_order_recency_norm',
              'department_order_recency','department_order_recency_norm'
             ]
print len(indep_vars), len(set(indep_vars))

157 157


In [7]:
final_train_data[final_train_data['user_product_num_streak'] > 2].reordered.mean()

0.0323626771569252

In [6]:
def get_kinetic_energy(x):
    sm = sum(x)
    p = [xi*1.0/sm for xi in x]
    return sum([pi**2 for pi in p]) 

var_imp = {}
for var in indep_vars:
    if final_train_data[var].nunique() < 200:
        var_imp[var] =  get_kinetic_energy(final_train_data[[var, 'reordered']].groupby(var).mean()['reordered'])
    else:
        a = final_train_data[[var, 'reordered']].sort_values(by = var)
        a['rank'] = range(a.shape[0])
        a['rank'] = a['rank'].map(lambda x : int(200*x/a.shape[0]))
        var_imp[var] = get_kinetic_energy(a[['rank', 'reordered']].groupby('rank').mean().reordered)
sorted([(k,v) for k,v in var_imp.items()], key = lambda x : x[1], reverse = True)

[('user_product_in_streak', 0.73652616497352619),
 ('is_gluten_free', 0.50460493293671682),
 ('is_organic', 0.50370606185670763),
 ('user_product_purchase_due', 0.50194573770315731),
 ('user_product_last_dow_diff', 0.25234218507804596),
 ('user_last_order_dow_diff', 0.25107838881807065),
 ('user_product_last_tod_diff', 0.2004771032922221),
 ('user_last_order_tod_diff', 0.20007308199790441),
 ('order_dow', 0.14314298081572793),
 ('user_product_last_dow', 0.1430416290217581),
 ('Best before', 0.13548917951729286),
 ('order_hour_of_day', 0.12510050744346518),
 ('user_product_last_tod', 0.12505380150406015),
 ('user_mean_product_name_length', 0.073986498574287074),
 ('user_department_count', 0.064168572432364165),
 ('user_product_orders_since_last_order', 0.052621136436144279),
 ('department_reorder_frequency', 0.050946530576353698),
 ('department_recency', 0.050946530576353698),
 ('department_dow_kinetic_energy', 0.050946530576353698),
 ('department_order_recency', 0.050946530576353691),


In [5]:
random.seed(2)
keep_set = set(random.sample(set(final_test_pre_final_data.order_id), 7500))

final_train_data = final_train_data.append(final_test_pre_final_data[final_test_pre_final_data.order_id.isin(keep_set)])
final_train_data.shape

(9106995, 132)

In [9]:
del final_test_pre_final_data

In [7]:
sorted([(var, final_train_data[[var, 'reordered']].corr()[var]['reordered']) for var in indep_vars], 
                     key = lambda x : x[1], reverse = True)
# final_train_data[['product_none_similarity', 'reordered']].corr()['product_none_similarity']['reordered']

[('user_product_purchase_ratio', 0.37012528871340616),
 ('user_product_max_streak_length', 0.27409302119880669),
 ('user_product_mean_streak_length', 0.24953556957570378),
 ('user_product_purchase_count', 0.24786927749400206),
 ('user_product_last_streak_length', 0.23294596399488735),
 ('posterior_probability_product', 0.19731083537334407),
 ('user_product_in_streak', 0.19697168543268212),
 ('user_product_hourly_reorder_rate', 0.19398279459410034),
 ('user_product_daily_reorder_rate', 0.19379849425799286),
 ('product_order_per_user', 0.18422120790341534),
 ('product_mean_streak_length', 0.1806656929159566),
 ('product_daily_reorder_rate', 0.14827954738217414),
 ('product_hourly_reorder_rate', 0.14817747786764743),
 ('product_fraction_users_with_no_reorders', 0.14589727229479832),
 ('posterior_probability_cluster', 0.13937722370693695),
 ('posterior_probability_aisle', 0.13360453553806428),
 ('cluster_reorder_frequency', 0.11755354774054581),
 ('product_order_count', 0.11463080963214053

In [6]:
corrs = final_train_data[indep_vars].corr()
for v1 in indep_vars:
    for v2 in indep_vars:
        if v1 != v2 and corrs[v1][v2] > 0.90:
            print v1, v2, corrs[v1][v2]

user_product_count user_num_product_with_no_reorder 0.929206542734
product_mean_streak_length product_order_per_user 0.95730620042
user_num_product_with_no_reorder user_product_count 0.929206542734
product_fraction_users_with_no_reorders product_daily_reorder_rate 0.902915418833
product_fraction_users_with_no_reorders product_hourly_reorder_rate 0.900670291945
user_cluster_mean_reorder_count user_cluster_mean_product_counts 0.947155622685
user_cluster_mean_product_counts user_cluster_mean_reorder_count 0.947155622685
product_daily_reorder_rate product_fraction_users_with_no_reorders 0.902915418833
product_daily_reorder_rate product_hourly_reorder_rate 0.906452059737
product_hourly_reorder_rate product_fraction_users_with_no_reorders 0.900670291945
product_hourly_reorder_rate product_daily_reorder_rate 0.906452059737
product_order_per_user product_mean_streak_length 0.95730620042
aisle_order_recency_norm department_order_recency_norm 0.930398970768
department_order_recency_norm aisle_or

In [17]:
sorted([(var, final_train_data[var].mean()/final_test_data[var].mean()) for var in indep_vars], key = lambda x : x[1])

[('aisle_user_count', 0.9212177397098749),
 ('aisle_dow_kinetic_energy', 0.9433180905899092),
 ('cluster_dow_kinetic_energy', 0.9434658000841912),
 ('department_dow_kinetic_energy', 0.9436781888984471),
 ('product_dow_kinetic_energy', 0.9499102173293186),
 ('user_mean_product_name_idf', 0.9507042003907913),
 ('aisle_order_recency_norm', 0.965251740691444),
 ('department_order_recency_norm', 0.9666028633622162),
 ('user_product_purchase_count', 0.9688596659587786),
 ('cluster_order_per_user', 0.9704464214774452),
 ('aisle_order_recency', 0.9718602121803435),
 ('user_mean_streak_length', 0.9721193040790348),
 ('cluster_user_count', 0.9728339189896344),
 ('department_order_recency', 0.9736492580924486),
 ('cluster_order_recency_norm', 0.9742494157152892),
 ('product_name_idf', 0.9753316800276204),
 ('cluster_reorder_frequency', 0.9793189719458406),
 ('product_mean_streak_length', 0.979582755345662),
 ('aisle_reorder_frequency', 0.9802836166599056),
 ('cluster_mean_add_to_cart_order', 0.98

In [6]:
final_train_data.user_product_num_streak = final_train_data.user_product_num_streak.map(lambda x : 0 if x == np.inf else x)
final_test_data.user_product_num_streak = final_test_data.user_product_num_streak.map(lambda x : 0 if x == np.inf else x)

In [7]:
final_train_data.user_product_daily_order_rate = final_train_data.user_product_daily_order_rate.map(lambda x : 0 if x == np.inf else x)
final_test_data.user_product_daily_order_rate = final_test_data.user_product_daily_order_rate.map(lambda x : 0 if x == np.inf else x)
final_train_data.user_product_hourly_order_rate = final_train_data.user_product_hourly_order_rate.map(lambda x : 0 if x == np.inf else x)
final_test_data.user_product_hourly_order_rate = final_test_data.user_product_hourly_order_rate.map(lambda x : 0 if x == np.inf else x)

In [20]:
final_train_data.user_product_num_streak.mean(), final_test_data.user_product_num_streak.mean()

(0.896084865623776, 0.8956980798904385)

In [20]:
random.seed(2)
dev_set = set(random.sample(set(final_train_data.order_id), int(len(set(final_train_data.order_id))*0.8)))
val_set = set(final_train_data.order_id) - dev_set
# val_set = set(valid1.user_id)
# dev_set = set(final_train_data.user_id) - val_set

# X_dev = final_train_data[final_train_data.user_id.isin(dev_set)][indep_vars]
# y_dev = final_train_data[final_train_data.user_id.isin(dev_set)]['reordered']
# X_val = final_train_data[final_train_data.user_id.isin(val_set)][indep_vars]
# y_val = final_train_data[final_train_data.user_id.isin(val_set)]['reordered']


X_dev = final_train_data[final_train_data.order_id.isin(dev_set)][indep_vars]
y_dev = final_train_data[final_train_data.order_id.isin(dev_set)]['reordered']
X_val = final_train_data[final_train_data.order_id.isin(val_set)][indep_vars]
y_val = final_train_data[final_train_data.order_id.isin(val_set)]['reordered']

lgb_dev = lgb.Dataset(X_dev, y_dev)
lgb_val = lgb.Dataset(X_val, y_val)

In [21]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective':'binary',
    'metric': {'auc', 'binary_logloss'},
    'num_leaves': 825,
    'learning_rate': 0.0152,
    'feature_fraction': 0.77,
    'bagging_fraction': 0.75,
    'bagging_freq': 1,
    'verbose': 1,
    'min_data_in_leaf' : 1800,
    'max_bin' : 255,
    'lambda_l1' : 0.000025,
    'lambda_l2' : 0.000025,
    'min_gain_to_split' : 0.00004
}

model = lgb.train(params, lgb_dev, num_boost_round = 3000, valid_sets = (lgb_val), early_stopping_rounds = 100,
                 verbose_eval = True)
pred_valid = model.predict(X_val)
np.mean(pred_valid)

[1]	valid_0's binary_logloss: 0.682419	valid_0's auc: 0.820933
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's binary_logloss: 0.671977	valid_0's auc: 0.826242
[3]	valid_0's binary_logloss: 0.661787	valid_0's auc: 0.829914
[4]	valid_0's binary_logloss: 0.651898	valid_0's auc: 0.831029
[5]	valid_0's binary_logloss: 0.642292	valid_0's auc: 0.831612
[6]	valid_0's binary_logloss: 0.632955	valid_0's auc: 0.831991
[7]	valid_0's binary_logloss: 0.623878	valid_0's auc: 0.832209
[8]	valid_0's binary_logloss: 0.615061	valid_0's auc: 0.832379
[9]	valid_0's binary_logloss: 0.606539	valid_0's auc: 0.832453
[10]	valid_0's binary_logloss: 0.598198	valid_0's auc: 0.83255
[11]	valid_0's binary_logloss: 0.590082	valid_0's auc: 0.832609
[12]	valid_0's binary_logloss: 0.582202	valid_0's auc: 0.832659
[13]	valid_0's binary_logloss: 0.574519	valid_0's auc: 0.832741
[14]	valid_0's binary_logloss: 0.567032	valid_0's auc: 0.832772
[15]	valid_0's binary_logloss: 0.55975	valid_0's au

[129]	valid_0's binary_logloss: 0.27113	valid_0's auc: 0.83507
[130]	valid_0's binary_logloss: 0.270527	valid_0's auc: 0.835087
[131]	valid_0's binary_logloss: 0.269933	valid_0's auc: 0.835115
[132]	valid_0's binary_logloss: 0.269354	valid_0's auc: 0.835134
[133]	valid_0's binary_logloss: 0.268789	valid_0's auc: 0.835154
[134]	valid_0's binary_logloss: 0.268244	valid_0's auc: 0.835172
[135]	valid_0's binary_logloss: 0.267739	valid_0's auc: 0.835186
[136]	valid_0's binary_logloss: 0.267213	valid_0's auc: 0.835192
[137]	valid_0's binary_logloss: 0.266689	valid_0's auc: 0.835209
[138]	valid_0's binary_logloss: 0.266185	valid_0's auc: 0.835237
[139]	valid_0's binary_logloss: 0.265689	valid_0's auc: 0.835242
[140]	valid_0's binary_logloss: 0.265197	valid_0's auc: 0.835268
[141]	valid_0's binary_logloss: 0.264716	valid_0's auc: 0.835286
[142]	valid_0's binary_logloss: 0.264251	valid_0's auc: 0.835308
[143]	valid_0's binary_logloss: 0.263822	valid_0's auc: 0.835323
[144]	valid_0's binary_logl

[256]	valid_0's binary_logloss: 0.243831	valid_0's auc: 0.83735
[257]	valid_0's binary_logloss: 0.243787	valid_0's auc: 0.837364
[258]	valid_0's binary_logloss: 0.243742	valid_0's auc: 0.837381
[259]	valid_0's binary_logloss: 0.243701	valid_0's auc: 0.837395
[260]	valid_0's binary_logloss: 0.243658	valid_0's auc: 0.837415
[261]	valid_0's binary_logloss: 0.243616	valid_0's auc: 0.837432
[262]	valid_0's binary_logloss: 0.243576	valid_0's auc: 0.837446
[263]	valid_0's binary_logloss: 0.243534	valid_0's auc: 0.837462
[264]	valid_0's binary_logloss: 0.243491	valid_0's auc: 0.837483
[265]	valid_0's binary_logloss: 0.243457	valid_0's auc: 0.837493
[266]	valid_0's binary_logloss: 0.243423	valid_0's auc: 0.837503
[267]	valid_0's binary_logloss: 0.243385	valid_0's auc: 0.837521
[268]	valid_0's binary_logloss: 0.243351	valid_0's auc: 0.837534
[269]	valid_0's binary_logloss: 0.243318	valid_0's auc: 0.837544
[270]	valid_0's binary_logloss: 0.243278	valid_0's auc: 0.837565
[271]	valid_0's binary_log

[383]	valid_0's binary_logloss: 0.24158	valid_0's auc: 0.83879
[384]	valid_0's binary_logloss: 0.241572	valid_0's auc: 0.838802
[385]	valid_0's binary_logloss: 0.241566	valid_0's auc: 0.83881
[386]	valid_0's binary_logloss: 0.241561	valid_0's auc: 0.838817
[387]	valid_0's binary_logloss: 0.241555	valid_0's auc: 0.838824
[388]	valid_0's binary_logloss: 0.241546	valid_0's auc: 0.838836
[389]	valid_0's binary_logloss: 0.24154	valid_0's auc: 0.838844
[390]	valid_0's binary_logloss: 0.241536	valid_0's auc: 0.838849
[391]	valid_0's binary_logloss: 0.241531	valid_0's auc: 0.838855
[392]	valid_0's binary_logloss: 0.241525	valid_0's auc: 0.838863
[393]	valid_0's binary_logloss: 0.241519	valid_0's auc: 0.838871
[394]	valid_0's binary_logloss: 0.241514	valid_0's auc: 0.838879
[395]	valid_0's binary_logloss: 0.241509	valid_0's auc: 0.838887
[396]	valid_0's binary_logloss: 0.241501	valid_0's auc: 0.838897
[397]	valid_0's binary_logloss: 0.241496	valid_0's auc: 0.838903
[398]	valid_0's binary_loglos

[510]	valid_0's binary_logloss: 0.241113	valid_0's auc: 0.839517
[511]	valid_0's binary_logloss: 0.241113	valid_0's auc: 0.839517
[512]	valid_0's binary_logloss: 0.241111	valid_0's auc: 0.83952
[513]	valid_0's binary_logloss: 0.241109	valid_0's auc: 0.839524
[514]	valid_0's binary_logloss: 0.241106	valid_0's auc: 0.83953
[515]	valid_0's binary_logloss: 0.241103	valid_0's auc: 0.839535
[516]	valid_0's binary_logloss: 0.241102	valid_0's auc: 0.839538
[517]	valid_0's binary_logloss: 0.241099	valid_0's auc: 0.839543
[518]	valid_0's binary_logloss: 0.241094	valid_0's auc: 0.839552
[519]	valid_0's binary_logloss: 0.241092	valid_0's auc: 0.839555
[520]	valid_0's binary_logloss: 0.24109	valid_0's auc: 0.83956
[521]	valid_0's binary_logloss: 0.241089	valid_0's auc: 0.839564
[522]	valid_0's binary_logloss: 0.241087	valid_0's auc: 0.839567
[523]	valid_0's binary_logloss: 0.241083	valid_0's auc: 0.839573
[524]	valid_0's binary_logloss: 0.241082	valid_0's auc: 0.839575
[525]	valid_0's binary_loglos

...

[891]	valid_0's binary_logloss: 0.240795	valid_0's auc: 0.840119
[892]	valid_0's binary_logloss: 0.240795	valid_0's auc: 0.840118
[893]	valid_0's binary_logloss: 0.240795	valid_0's auc: 0.840119
[894]	valid_0's binary_logloss: 0.240794	valid_0's auc: 0.840121
[895]	valid_0's binary_logloss: 0.240794	valid_0's auc: 0.840121
[896]	valid_0's binary_logloss: 0.240796	valid_0's auc: 0.840117
[897]	valid_0's binary_logloss: 0.240797	valid_0's auc: 0.840116
[898]	valid_0's binary_logloss: 0.240797	valid_0's auc: 0.840116
[899]	valid_0's binary_logloss: 0.240796	valid_0's auc: 0.840116
[900]	valid_0's binary_logloss: 0.240795	valid_0's auc: 0.840117
[901]	valid_0's binary_logloss: 0.240796	valid_0's auc: 0.840116
[902]	valid_0's binary_logloss: 0.240795	valid_0's auc: 0.840119
[903]	valid_0's binary_logloss: 0.240794	valid_0's auc: 0.840119
[904]	valid_0's binary_logloss: 0.240793	valid_0's auc: 0.840121
[905]	valid_0's binary_logloss: 0.240793	valid_0's auc: 0.840122
[906]	valid_0's binary_lo

[1018]	valid_0's binary_logloss: 0.240769	valid_0's auc: 0.840169
[1019]	valid_0's binary_logloss: 0.240768	valid_0's auc: 0.84017
[1020]	valid_0's binary_logloss: 0.240769	valid_0's auc: 0.84017
[1021]	valid_0's binary_logloss: 0.240769	valid_0's auc: 0.840169
[1022]	valid_0's binary_logloss: 0.240768	valid_0's auc: 0.840169
[1023]	valid_0's binary_logloss: 0.240766	valid_0's auc: 0.840172
[1024]	valid_0's binary_logloss: 0.240766	valid_0's auc: 0.840173
[1025]	valid_0's binary_logloss: 0.240766	valid_0's auc: 0.840174
[1026]	valid_0's binary_logloss: 0.240765	valid_0's auc: 0.840175
[1027]	valid_0's binary_logloss: 0.240766	valid_0's auc: 0.840172
[1028]	valid_0's binary_logloss: 0.240768	valid_0's auc: 0.840169
[1029]	valid_0's binary_logloss: 0.240768	valid_0's auc: 0.84017
[1030]	valid_0's binary_logloss: 0.240768	valid_0's auc: 0.840169
[1031]	valid_0's binary_logloss: 0.240769	valid_0's auc: 0.840169
[1032]	valid_0's binary_logloss: 0.240768	valid_0's auc: 0.84017
[1033]	valid_0

[1143]	valid_0's binary_logloss: 0.240758	valid_0's auc: 0.840188
[1144]	valid_0's binary_logloss: 0.240758	valid_0's auc: 0.840187
[1145]	valid_0's binary_logloss: 0.240759	valid_0's auc: 0.840186
[1146]	valid_0's binary_logloss: 0.240759	valid_0's auc: 0.840186
[1147]	valid_0's binary_logloss: 0.240758	valid_0's auc: 0.840187
[1148]	valid_0's binary_logloss: 0.240759	valid_0's auc: 0.840185
[1149]	valid_0's binary_logloss: 0.240759	valid_0's auc: 0.840185
[1150]	valid_0's binary_logloss: 0.240759	valid_0's auc: 0.840187
[1151]	valid_0's binary_logloss: 0.240758	valid_0's auc: 0.840187
[1152]	valid_0's binary_logloss: 0.240758	valid_0's auc: 0.840187
[1153]	valid_0's binary_logloss: 0.240758	valid_0's auc: 0.840186
[1154]	valid_0's binary_logloss: 0.240758	valid_0's auc: 0.840187
[1155]	valid_0's binary_logloss: 0.240757	valid_0's auc: 0.840189
[1156]	valid_0's binary_logloss: 0.240758	valid_0's auc: 0.840189
[1157]	valid_0's binary_logloss: 0.240759	valid_0's auc: 0.840186
[1158]	val

[1268]	valid_0's binary_logloss: 0.240763	valid_0's auc: 0.840186
[1269]	valid_0's binary_logloss: 0.240764	valid_0's auc: 0.840183
[1270]	valid_0's binary_logloss: 0.240764	valid_0's auc: 0.840183
[1271]	valid_0's binary_logloss: 0.240764	valid_0's auc: 0.840184
[1272]	valid_0's binary_logloss: 0.240764	valid_0's auc: 0.840183
[1273]	valid_0's binary_logloss: 0.240764	valid_0's auc: 0.840182
[1274]	valid_0's binary_logloss: 0.240764	valid_0's auc: 0.840182
[1275]	valid_0's binary_logloss: 0.240765	valid_0's auc: 0.840181
[1276]	valid_0's binary_logloss: 0.240763	valid_0's auc: 0.840184
[1277]	valid_0's binary_logloss: 0.240763	valid_0's auc: 0.840184
[1278]	valid_0's binary_logloss: 0.240764	valid_0's auc: 0.840184
[1279]	valid_0's binary_logloss: 0.240763	valid_0's auc: 0.840185
[1280]	valid_0's binary_logloss: 0.240762	valid_0's auc: 0.840186
[1281]	valid_0's binary_logloss: 0.240764	valid_0's auc: 0.840184
[1282]	valid_0's binary_logloss: 0.240763	valid_0's auc: 0.840185
[1283]	val

0.096046834606188028

In [12]:
sorted([(k,v) for k,v in zip(indep_vars, model.feature_importance())], key = lambda x : x[1], reverse = True)

[('days_since_prior_order', 16580),
 ('user_product_recency', 14330),
 ('user_mean_streak_length', 13226),
 ('user_order_frequency_std', 12560),
 ('user_last_2_order_sim', 12500),
 ('product_recency_norm', 12146),
 ('user_fraction_product_with_no_reorder', 11885),
 ('user_order_recency', 11847),
 ('user_organic_recency', 11816),
 ('user_hourly_reorder_rate', 11761),
 ('user_product_purchase_ratio', 11457),
 ('user_product_similarity', 11381),
 ('user_daily_reorder_rate', 11115),
 ('user_organic_fraction', 10886),
 ('user_last_n_order_sim', 10670),
 ('user_product_orders_since_last_order', 10357),
 ('user_reorder_count_std', 9664),
 ('user_cluster_reorder_rate', 9570),
 ('user_reorder_fraction_std', 9559),
 ('user_std_product_name_idf', 9502),
 ('user_std_basket_size', 9398),
 ('user_reorder_fraction_mean', 9101),
 ('user_std_product_name_length', 9093),
 ('user_gluten_free_recency', 8736),
 ('product_daily_order_rate', 8465),
 ('user_cluster_kinetic_energy', 8390),
 ('user_order_recenc

In [13]:
sorted([(k,v) for k,v in zip(indep_vars, model.feature_importance(importance_type='gain'))], key = lambda x : x[1], reverse = True)

[('user_product_purchase_ratio', 8552018.424985949),
 ('user_product_orders_since_last_order', 6862234.679468709),
 ('user_product_num_streak', 1292680.243432276),
 ('user_product_recency', 859804.93214725656),
 ('days_since_prior_order', 634525.12973748345),
 ('user_product_days_since_last_order', 585012.50561519666),
 ('product_mean_streak_length', 538217.17935164867),
 ('product_fraction_users_with_no_reorders', 491321.83620111435),
 ('user_product_purchase_count', 489465.26012298156),
 ('user_product_max_streak_length', 365091.20811862987),
 ('user_mean_streak_length', 327263.3429025212),
 ('user_product_mean_streak_length', 326910.97257388686),
 ('user_product_mean_days_between_purchase', 290102.48264181323),
 ('user_product_recency_order', 280960.14296307199),
 ('posterior_probability_product', 275495.46756064758),
 ('user_product_std_days_between_purchase', 273013.00972840638),
 ('user_order_recency', 254209.62510273099),
 ('product_recency_norm', 227033.4668164317),
 ('user_fra

In [22]:
valid = final_train_data[final_train_data.order_id.isin(val_set)][['order_id', 'product_id', 'reordered']]
valid['pred'] = pred_valid
valid['reordered'].mean()

0.09738201647996902

In [23]:
valid_t = valid.copy().reset_index(drop = True)
valid_t['rank'] = valid_t[['order_id', 'pred']].groupby('order_id').rank(ascending = False)
order_maxrank = {row[0]: row[1] for row in valid_t[['order_id', 'reordered']].groupby('order_id').sum().itertuples()}
valid_t['trim'] = valid_t.apply(lambda row : 1.0 if row['rank'] > order_maxrank[row['order_id']]*1.35
                                else 0.0, axis = 1)
valid_t = valid_t[valid_t['trim'] == 0]
# valid_t = valid_t[valid_t['rank'] < 23]
tp = valid_t[['order_id', 'reordered']].groupby('order_id').sum().reset_index()
tp.columns = ['order_id', 'tp']
num_pred = valid_t[['order_id', 'reordered']].groupby('order_id').count().reset_index()
num_pred.columns = ['order_id', 'num_pred']
num_act = valid[['order_id', 'reordered']].groupby('order_id').sum().reset_index()
num_act.columns = ['order_id', 'num_act']

valid_final = num_act.merge(num_pred, on = 'order_id', how = 'left').merge(tp, on = 'order_id', how = 'left')
valid_final.fillna(0, inplace = True)
def get_f1score(row):
    if row['tp'] == 0:
        return 0.0
    p = row['tp']/row['num_pred']
    r = row['tp']/row['num_act']
    return (2*p*r)/(p+r)
valid_final['f1_score'] = valid_final.apply(lambda row : get_f1score(row), axis = 1)
valid_final['f1_score'].describe()

count    26242.000000
mean         0.412754
std          0.298177
min          0.000000
25%          0.181818
50%          0.444444
75%          0.571429
max          1.000000
Name: f1_score, dtype: float64

In [24]:
valid_t = valid.copy().reset_index(drop = True)
valid_t['rank'] = valid_t[['order_id', 'pred']].groupby('order_id').rank(ascending = False)
order_maxrank = {row[0]: row[1] for row in valid_t[['order_id', 'pred']].groupby('order_id').sum().itertuples()}
valid_t['trim'] = valid_t.apply(lambda row : 1.0 if row['rank'] > order_maxrank[row['order_id']]*1.55
                                else 0.0, axis = 1)
valid_t = valid_t[valid_t['trim'] == 0]
# valid_t = valid_t[valid_t['rank'] < 23]
tp = valid_t[['order_id', 'reordered']].groupby('order_id').sum().reset_index()
tp.columns = ['order_id', 'tp']
num_pred = valid_t[['order_id', 'reordered']].groupby('order_id').count().reset_index()
num_pred.columns = ['order_id', 'num_pred']
num_act = valid[['order_id', 'reordered']].groupby('order_id').sum().reset_index()
num_act.columns = ['order_id', 'num_act']

valid_final = num_act.merge(num_pred, on = 'order_id', how = 'left').merge(tp, on = 'order_id', how = 'left')
valid_final.fillna(0, inplace = True)
def get_f1score(row):
    if row['tp'] == 0:
        return 0.0
    p = row['tp']/row['num_pred']
    r = row['tp']/row['num_act']
    return (2*p*r)/(p+r)
valid_final['f1_score'] = valid_final.apply(lambda row : get_f1score(row), axis = 1)
valid_final['f1_score'].describe()

count    26242.000000
mean         0.394132
std          0.253475
min          0.000000
25%          0.222222
50%          0.400000
75%          0.571429
max          1.000000
Name: f1_score, dtype: float64

In [8]:
# def get_expected_f1score(prediction, scores):
#     expected_f1score = 0
#     for i in range(1, len(scores) + 1):
#         for c in itertools.combinations(scores.keys(), i):
#             c = set(c)
#             prob = 1.0
#             for p in scores.keys():
#                 if p in c:
#                     prob *= scores[p]
#                 else:
#                     prob *= 1.0 - scores[p]
#             precision = 1.0*len(prediction.intersection(c))/len(c)
#             recall = 1.0*len(prediction.intersection(c))/len(prediction)
#             if precision*recall == 0:
#                 f1score = 0.0
#             else:
#                 f1score = 2*precision*recall/(precision+recall)
#             expected_f1score += prob*f1score
#     return expected_f1score

def get_expected_f1score(prediction, scores):
    expected_f1score = 0
    if len(prediction) == 0:
        return expected_f1score
#     scores = {k:v for k,v in scores.items() if v > 0.20}
    for i in xrange(1, len(scores) + 1):
        product_set = set(scores.keys()) - {0.0}
        for c in itertools.combinations(product_set, i):
            c = set(c)
            common = prediction.intersection(c)
            if len(common) == 0:
                continue
            precision = 1.0*len(common)/len(c)
            recall = 1.0*len(common)/len(prediction)
            if precision*recall == 0:
                f1score = 0.0
            else:
                f1score = 2*precision*recall/(precision+recall)
            prob = 1.0
            for p in scores.keys():
                if p in c:
                    prob *= scores[p]
                else:
                    prob *= 1.0 - scores[p]
            expected_f1score += prob*f1score

    c = {0.0}
    prob = 1.0
    for p in scores.keys():
        if p in c:
            prob *= scores[p]
#         else:
#             prob *= 1.0 - scores[p]
    precision = 1.0*len(prediction.intersection(c))/len(c)
    recall = 1.0*len(prediction.intersection(c))/len(prediction)
    if precision*recall == 0:
        f1score = 0.0
    else:
        f1score = 2*precision*recall/(precision+recall)
    expected_f1score += prob*f1score
    return expected_f1score

# def get_best_predictions(scores):
#     best_prediction = set()
#     best_score = 0.0
#     for i in range(1, len(scores) + 1):
#         improvement = False
#         for c in [set(c) for c in itertools.combinations(scores.keys(), i) 
#                   if best_prediction.intersection(set(c)) == best_prediction]:
#             prediction = c
#             score = get_expected_f1score(prediction, scores)
#             if score > best_score:
#                 best_prediction = prediction
#                 best_score = score
#                 improvement = True
#         if not improvement:
#             break
        
#     return [best_prediction, best_score]

# def get_best_predictions(scores):
#     best_prediction = set()
#     best_score = 0.0
#     for i in xrange(1, len(scores) + 1):
#         improvement = False
#         for c in [set([p for p,s in sorted([(p,s) for p,s in scores.items()], 
#                                            key = lambda x : x[1], reverse = True)[:i]])]:
#             prediction = c
#             score = get_expected_f1score(prediction, scores)
#             if score > best_score:
#                 best_prediction = prediction
#                 best_score = score
#                 improvement = True
#         if not improvement:
#             break
        
#     return [best_prediction, best_score]

def get_best_predictions(scores):
    i = (1*len(scores))/2 + 1
    best_prediction = set([p for p,s in sorted([(p,s) for p,s in scores.items()], 
                                              key = lambda x : x[1], reverse = True)[:i]])
    best_score = get_expected_f1score(best_prediction, scores)   
    done = False
    go_right = True
    num_step = 0
    while not done:
        if go_right:
            i +=1
        else:
            i -= 1
        num_step += 1
        prediction = set([p for p,s in sorted([(p,s) for p,s in scores.items()], 
                                              key = lambda x : x[1], reverse = True)[:i]])
        score = get_expected_f1score(prediction, scores)
        if score > best_score:
            best_prediction = prediction
            best_score = score
        else:
            if go_right:
                if num_step == 1:
                    i = (1*len(scores))/2 + 1
                    go_right = False
                else:
                    done = True
            else:
                done = True
        
    return [best_prediction, best_score]



def get_best_predictions1(scores, sim_size = 13):
    if len(scores) > sim_size:
        best_prediction = set()
        best_score = 0.0
        for j in range(0, len(scores) - sim_size):
            [bp,bs] = get_best_predictions1({p:s for p,s in sorted([(p,s) for p,s in scores.items()], 
                                              key = lambda x : x[1], reverse = True)[j:sim_size + j]}, sim_size)
            best_prediction = best_prediction.union(bp)
            best_score = bs
            if len(bp) < sim_size - 5:
                return [best_prediction, best_score]
        return [best_prediction, best_score]
    i = (1*len(scores))/2 + 1
    best_prediction = set([p for p,s in sorted([(p,s) for p,s in scores.items()], 
                                              key = lambda x : x[1], reverse = True)[:i]])
    best_score = get_expected_f1score(best_prediction, scores)   
    done = False
    go_right = True
    num_step = 0
    while not done:
        if go_right:
            i +=1
        else:
            i -= 1
        num_step += 1
        prediction = set([p for p,s in sorted([(p,s) for p,s in scores.items()], 
                                              key = lambda x : x[1], reverse = True)[:i]])
        score = get_expected_f1score(prediction, scores)
        if score > best_score:
            best_prediction = copy.deepcopy(prediction)
            best_score = score
        else:
            if go_right:
                if num_step == 1:
                    i = (1*len(scores))/2 + 1
                    go_right = False
                else:
                    done = True
            else:
                done = True
        
    return [best_prediction, best_score]

In [9]:
class F1Optimizer():
    def __init__(self):
        pass

    @staticmethod
    def get_expectations(P, pNone=None):
        expectations = []
        P = np.sort(P)[::-1]

        n = np.array(P).shape[0]
        DP_C = np.zeros((n + 2, n + 1))
        if pNone is None:
            pNone = (1.0 - P).prod()

        DP_C[0][0] = 1.0
        for j in range(1, n):
            DP_C[0][j] = (1.0 - P[j - 1]) * DP_C[0, j - 1]

        for i in range(1, n + 1):
            DP_C[i, i] = DP_C[i - 1, i - 1] * P[i - 1]
            for j in range(i + 1, n + 1):
                DP_C[i, j] = P[j - 1] * DP_C[i - 1, j - 1] + (1.0 - P[j - 1]) * DP_C[i, j - 1]

        DP_S = np.zeros((2 * n + 1,))
        DP_SNone = np.zeros((2 * n + 1,))
        for i in range(1, 2 * n + 1):
            DP_S[i] = 1. / (1. * i)
            DP_SNone[i] = 1. / (1. * i + 1)
        for k in range(n + 1)[::-1]:
            f1 = 0
            f1None = 0
            for k1 in range(n + 1):
                f1 += 2 * k1 * DP_C[k1][k] * DP_S[k + k1]
                f1None += 2 * k1 * DP_C[k1][k] * DP_SNone[k + k1]
            for i in range(1, 2 * k - 1):
                DP_S[i] = (1 - P[k - 1]) * DP_S[i] + P[k - 1] * DP_S[i + 1]
                DP_SNone[i] = (1 - P[k - 1]) * DP_SNone[i] + P[k - 1] * DP_SNone[i + 1]
            expectations.append([f1None + 2 * pNone / (2 + k), f1])

        return np.array(expectations[::-1]).T

    @staticmethod
    def maximize_expectation(P, pNone=None):
        expectations = F1Optimizer.get_expectations(P, pNone)

        ix_max = np.unravel_index(expectations.argmax(), expectations.shape)
        max_f1 = expectations[ix_max]

        predNone = True if ix_max[0] == 0 else False
        best_k = ix_max[1]

        return best_k, predNone, max_f1

    @staticmethod
    def _F1(tp, fp, fn):
        return 2 * tp / (2 * tp + fp + fn)

    @staticmethod
    def _Fbeta(tp, fp, fn, beta=1.0):
        beta_squared = beta ** 2
        return (1.0 + beta_squared) * tp / ((1.0 + beta_squared) * tp + fp + beta_squared * fn)

def get_best_prediction(scores):
    if 0.0 in scores.keys():
        pNone = scores[0.0]
    else:
        pNone = 0.0
    sorted_scores = sorted([(p,s) for p,s in scores.items()], key = lambda x : x[1], reverse = True)
    L = [p for p,s in sorted_scores if p != 0.0]
    P = [s for p,s in sorted_scores if p != 0.0]
    
#     P = np.sort(P)[::-1]
#     n = P.shape[0]
#     L = ['L{}'.format(i + 1) for i in range(n)]

#     if pNone is None:
#         print("Estimate p(None|x) as (1-p_1)*(1-p_2)*...*(1-p_n)")
#         pNone = (1.0 - P).prod()

#     PL = ['p({}|x)={}'.format(l, p) for l, p in zip(L, P)]
#     print("Posteriors: {} (n={})".format(PL, n))
#     print("p(None|x)={}".format(pNone))

    opt = F1Optimizer.maximize_expectation(P, pNone)
    best_prediction = [0.0] if opt[1] else []
    best_prediction += (L[:opt[0]])
    best_score = opt[2]
    
    return [best_prediction, best_score]

In [27]:
scores = {"A" : 0.5, "B" : 0.29, "C" : 0.35, "D" : 0.41, "E" : 0.37, "F" : 0.15, "G" : 0.22, "H" : 0.32, "I" : 0.18,
         "J" : 0.7, "K" : 0.5, "L" : 0.23, "M" : 0.25, "N" : 0.27, "M" : 0.30, "O" : 0.321, "P" : 0.41}
start = time.time()
print get_best_predictions(scores)
print "time taken", time.time() - start
start = time.time()
print get_best_predictions1(scores, 10)
print "time taken", time.time() - start
start = time.time()
print get_best_prediction(scores)
print "time taken", time.time() - start

[set(['A', 'C', 'B', 'E', 'D', 'H', 'K', 'J', 'M', 'O', 'N', 'P']), 0.5291751421481377]
time taken 1.08506512642
[set(['A', 'C', 'B', 'E', 'D', 'G', 'H', 'K', 'J', 'M', 'L', 'O', 'N', 'P']), 0.4306246540584367]
time taken 0.0885429382324
[['J', 'A', 'K', 'D', 'P', 'E', 'C', 'O', 'H', 'M', 'B', 'N'], 0.52917514214813699]
time taken 0.00230503082275


In [36]:
valid_t = valid.copy().reset_index(drop = True)
# valid_t = valid_t.merge(valid1[['user_id', 'product_id', 'pred1']], on = ['user_id', 'product_id'], how = 'inner')
# valid_t['pred'] = valid_t.pred/2 + valid_t.pred1/2
valid_t = valid_t[['order_id', 'product_id','pred', 'reordered']].groupby('order_id').aggregate(
        lambda x : tuple(x)).reset_index()
valid_t['pred1'] = valid_t.apply(lambda row : {p:s for p,s in sorted(zip(row['product_id'],row['pred']),
                                    key = lambda x : x[1], reverse = True)[:min(int(np.round(sum(row['pred']) + 6.0 +
                                                70.0*np.log(1 + sum(row['pred'])))), len(row['pred']))]}, axis = 1)
valid_t['pred2'] = valid_t.pred1.map(lambda x : {p:s for p,s in sorted(x.items(), 
                                                                       key = lambda x : x[1])[:min(20, len(x))]})
valid_t['pred3'] = valid_t.pred1.map(lambda x : {p for p,s in sorted(x.items(), 
                                                                     key = lambda x : x[1])[min(20, len(x)):]})
valid_t['actual'] = valid_t.apply(lambda row : {p for p,a in zip(row['product_id'],row['reordered'])
                                                if a == 1}, axis = 1)

In [38]:
valid_t = valid.copy().reset_index(drop = True)
valid_t = valid_t[['order_id', 'product_id','pred', 'reordered']].groupby('order_id').aggregate(
        lambda x : tuple(x)).reset_index()
valid_t['pred1'] = valid_t.apply(lambda row : {p:s for p,s in sorted(zip(row['product_id'],row['pred']),
                                    key = lambda x : x[1], reverse = True)[:min(int(np.round(sum(row['pred'])*5)), 
                                                                                len(row['pred']))]}, axis = 1)
# valid_t['pred2'] = valid_t.pred1.map(lambda x : {p:s for p,s in sorted(x.items(), 
#                                                                        key = lambda x : x[1])[:min(20, len(x))]})
# valid_t['pred3'] = valid_t.pred1.map(lambda x : {p for p,s in sorted(x.items(), 
#                                                                      key = lambda x : x[1])[min(20, len(x)):]})
valid_t['actual'] = valid_t.apply(lambda row : {p for p,a in zip(row['product_id'],row['reordered'])
                                                if a == 1}, axis = 1)

In [31]:
print valid_t.apply(lambda row : sum([p for p,s in sorted(zip(row['reordered'],row['pred']),
                                    key = lambda x : x[1], reverse = True)[:min(int(np.round(sum(row['pred']) + 600.0 +
                                            90.0*np.log(1 + sum(row['pred'])))), len(row['pred']))]]), axis = 1).sum()

166711.0


In [26]:
valid.reordered.sum()

166711.0

In [37]:
valid_t1 = valid_t.copy()
start = time.time()
pool = mp.Pool(processes=50)
valid_t1['predictions'] = pool.map(get_best_prediction,valid_t1.pred1)
pool.close()
pool.join()
print 'time taken:', time.time() - start

time taken: 37.3285079002


In [38]:
# valid_t1['final_prediction'] = valid_t1.apply(lambda row : row['pred3'].union(row['predictions'][0]), axis = 1)
valid_t1['final_prediction'] = valid_t1['predictions'].map(lambda x : set(x[0]))

def get_f1score(row):
    pred = row['final_prediction']
    actual = row['actual']
    if len(pred) == 0:
        pred = {0}
    p = len(pred.intersection(actual))*1.0/len(pred)
    r = len(pred.intersection(actual))*1.0/len(actual)
    if p*r == 0:
        return 0.0
    else:
        return 2*p*r/(p+r)
print valid_t1.apply(get_f1score, axis = 1).mean(), valid_t1['predictions'].map(lambda x : x[1]).mean()

0.404581114609 0.411632000298


In [35]:
valid_t1['has_none'] = valid_t1.final_prediction.map(lambda x : 1.0 if 0.0 in x else 0.0)
valid_t1.final_prediction = valid_t1.final_prediction.map(lambda x : x - {0.0} if 0.0 in x and len(x) > 21 else x)
valid_t1['weakest_prediction'] = valid_t1.apply(lambda row : sorted([(p,s) for p,s in zip(row['product_id'],row['pred'])
                                                                            if p in row['final_prediction']], key = lambda x : x[1])[0], axis = 1)
valid_t1.final_prediction = valid_t1.apply(lambda row : row['final_prediction']  - set([row['weakest_prediction'][0]]) if
                                           len(row['final_prediction']) > 20 and row['weakest_prediction'][1] < 0.09 else row['final_prediction'], axis = 1)
print valid_t1.apply(get_f1score, axis = 1).mean(), valid_t1['predictions'].map(lambda x : x[1]).mean()

0.404588374862 0.411515821027


In [34]:
valid_t1['f1score'] = valid_t1.apply(get_f1score, axis = 1)
valid_t1['len_actual'] = valid_t1.actual.map(len)
valid_t1['sum_pred'] = valid_t1.pred.map(lambda x : min(int(np.round(sum(x) + 6.0 +
                                                70.0*np.log(1 + sum(x)))), len(x)))

print valid_t1[valid_t1['sum_pred']  < 40].f1score.mean(), valid_t1[valid_t1['sum_pred']  < 40].f1score.count()
print valid_t1[valid_t1['sum_pred']  >= 40].f1score.mean(), valid_t1[valid_t1['sum_pred']  >= 40].f1score.count()


0.443901873708 10837
0.377985106405 15405


In [10]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective':'binary',
    'metric': {'auc', 'binary_logloss'},
    'num_leaves': 825,
    'learning_rate': 0.0152,
    'feature_fraction': 0.77,
    'bagging_fraction': 0.75,
    'bagging_freq': 1,
    'verbose': 1,
    'min_data_in_leaf' : 1750,
    'max_bin' : 255,
    'lambda_l1' : 0.000025,
    'lambda_l2' : 0.000025,
    'min_gain_to_split' : 0.00004
}
lgb_train = lgb.Dataset(final_train_data[indep_vars], final_train_data['reordered'])
model = lgb.train(params, lgb_train, num_boost_round = 1195)

In [11]:
pred = model.predict(final_test_data[indep_vars])
print pred.mean()
final_test_data['pred'] = pred

0.0970256102105


In [14]:
scores = pd.read_csv("../Data/arb_scores_with_none.csv")
scores.shape, final_test_data.shape

((4908292, 3), (4908292, 178))

In [15]:
scores.prediction.mean()

0.09721305960929008

In [16]:
final_test_data.pred.mean()

0.09702561021045665

In [17]:
submission = pd.read_csv("../Data/sample_submission.csv")
final_test_data_t = final_test_data.copy()[['order_id','product_id','pred']]
final_test_data_t = final_test_data_t.merge(scores, on = ['order_id','product_id'])
final_test_data_t.pred = final_test_data_t.apply(lambda row : np.sqrt(0.25*row['prediction']**2 + 0.75*row['pred']**2), axis = 1)
final_test_data_t = final_test_data_t[['order_id', 'product_id','pred']].groupby('order_id').aggregate(lambda x : 
                                                                        tuple(x)).reset_index()
final_test_data_t['pred1'] = final_test_data_t.apply(lambda row : {p:s for p,s in sorted(zip(row['product_id'],row['pred']),
                                    key = lambda x : x[1], reverse = True)[:min(int(np.round(sum(row['pred']) + 6 +
                                                90*np.log(1 + sum(row['pred'])))), len(row['pred']))]}, axis = 1)
# final_test_data_t['pred2'] = final_test_data_t.pred1.map(lambda x : {p:s for p,s in sorted(x.items(), key = 
#                                                 lambda x : x[1])[:min(15, len(x))]})
# final_test_data_t['pred3'] = final_test_data_t.pred1.map(lambda x : {p for p,s in sorted(x.items(), key = 
#                                         lambda x : x[1])[min(15, len(x)):]})

In [15]:
submission = pd.read_csv("../Data/sample_submission.csv")
final_test_data_t = final_test_data.copy()[['order_id','product_id','pred']]
final_test_data_t = final_test_data_t[['order_id', 'product_id','pred']].groupby('order_id').aggregate(lambda x : 
                                                                        tuple(x)).reset_index()
final_test_data_t['pred1'] = final_test_data_t.apply(lambda row : {p:s for p,s in sorted(zip(row['product_id'],row['pred']),
                                    key = lambda x : x[1], reverse = True)[:min(int(np.round(sum(row['pred'])*5)), 
                                                                                len(row['pred']))]}, axis = 1)


In [20]:
start = time.time()
pool = mp.Pool(processes=50)
final_test_data_t['predictions'] = pool.map(get_best_prediction,final_test_data_t.pred1)
pool.close()
pool.join()
print 'time taken:', time.time() - start

time taken: 102.756515026


In [21]:
# final_test_data_t['final_prediction'] = final_test_data_t.apply(lambda row : row['pred3'].union(row['predictions'][0]),
#                                                                 axis = 1)
final_test_data_t['final_prediction'] = final_test_data_t['predictions'].map(lambda x : set(x[0]))
final_test_data_t.final_prediction = final_test_data_t.final_prediction.map(lambda x : x - {0.0} if 0.0 in x and len(x) > 21 else x)
final_test_data_t['weakest_prediction'] = final_test_data_t.apply(lambda row : sorted([(p,s) for p,s in zip(row['product_id'],row['pred'])
                                                                            if p in row['final_prediction']], key = lambda x : x[1])[0], axis = 1)
final_test_data_t.final_prediction = final_test_data_t.apply(lambda row : row['final_prediction']  - set([row['weakest_prediction'][0]]) if
                                           len(row['final_prediction']) > 20 and row['weakest_prediction'][1] < 0.09 else row['final_prediction'], axis = 1)

def dump_product(x):
    if x == 0:
        return "None"
    else:
        return str(int(x))
final_test_data_t['products'] = final_test_data_t['final_prediction'].map(
                                                            lambda x : " ".join([dump_product(p) for p in x]))

In [22]:
del submission['products']
submission = submission.merge(final_test_data_t[['order_id', 'products']], on = 'order_id', how = 'left')
submission.fillna('None', inplace = True)
submission.products = submission.products.map(lambda x : "None" if x == "" else x)

In [23]:
submission.to_csv("./submission_lgb.csv", index = False)

In [19]:
submission.to_csv("./submission_localcv_0.403821266328  .csv", index = False)

In [ ]:
submission = pd.read_csv("../Data/sample_submission.csv")
final_test_data_t = final_test_data.copy()
final_test_data_t['rank'] = final_test_data_t[['order_id', 'pred']].groupby('order_id').rank(ascending = False)
order_maxrank = {row[0]: row[1] for row in final_test_data_t[['order_id', 'pred']].groupby('order_id').sum().itertuples()}

final_test_data_t['keep'] = final_test_data_t.apply(lambda row:1.0 if row['rank'] < order_maxrank[row['order_id']]*1.5
                                else 0.0, axis = 1)
final_test_data_t = final_test_data_t[final_test_data_t.keep == 1]

output = final_test_data_t[['order_id', 'product_id']].groupby('order_id').aggregate(
    lambda x : tuple(x)).reset_index()
def dump_product(x):
    if x == 0:
        return "None"
    else:
        return str(int(x))
output['products'] = output['product_id'].map(lambda x : " ".join([dump_product(p) for p in x]))

del submission['products']
submission = submission.merge(output[['order_id', 'products']], on = 'order_id', how = 'left')
submission.fillna('None', inplace = True)

submission.to_csv("./submission_localcv_0.401930972339.csv", index = False)